In [3]:
#%pylab inline
import os, sys, gzip, random, csv, json, datetime, re
import numpy as np
import pandas as pd
import scipy as sp
sys.path.append(os.environ['LAV_DIR']+'/src/')
baseDir = os.environ['LAV_DIR']
from math import sin, cos, sqrt, atan2, radians
import folium
import matplotlib.pyplot as plt
import geopandas as gpd
import pymongo
from scipy.spatial import cKDTree
from scipy import inf
import shapely as sh
from shapely.ops import split, snap
from shapely import geometry, ops
#with open('C:/Users/Maurice/Desktop/Jupyter/Nissan/geomadi_pub.json') as f:
#    cred = json.load(f)
#baseDir = "Z:/Data_Science/Customer_Projects_DE/Nissan/Germany_Graph/"
#client = pymongo.MongoClient(cred['mongo']['address'],cred['mongo']['port'])

%matplotlib inline

In [4]:
metr = {"metrics": {"gradMeter":111122.19769899677  ,"deCenter":[10.28826401,51.13341344]  ,"deBBox":[5.866,47.2704,15.0377,55.0574] }}

In [5]:
print('starting x boost junct collider alpha^2')
nodJ = gpd.read_file(baseDir + "motorway_link_nodes.geojson")
motG = gpd.GeoDataFrame.from_file(baseDir + "motorway.shp")
motL = motG.geometry.unary_union
junG = gpd.read_file(baseDir + "de_junct.shp")
jun_schwer = pd.read_csv(baseDir + "de_junct_unique.csv")
nodJ.loc[:,"node_id"] = nodJ['node_id'].astype(int)
print("files loaded")

starting x boost junct collider alpha^2


OSError: no such file or directory: '/home/sabeiro/lav/motion/motorway_link_nodes.geojson'

# Test with Junction A4-53

In [301]:
Test_53 = nodJ.loc[nodJ['id_jun'] == "A 4-53"]
Test_53 = pd.merge(Test_53, jun_schwer, left_on=["id_jun"], right_on=["id_jun"], how = "left")
Test_53 = Test_53.drop(["field_7", "field_5", "field_6"], axis = 1)
Test_53.head()

,id,id_jun,lat,node_id,destination,lon,geometry,x,y
0,218436,A 4-53,50.871500,1478289457,B 88,11.593074,POINT (11.5930741 50.8715002),11.594058,50.872094
1,218495,A 4-53,50.871972,486611,A 4,11.593503,POINT (11.5935028 50.8719723),11.594058,50.872094
2,218504,A 4-53,50.871613,1788468207,B 88,11.593551,POINT (11.5935514 50.8716134),11.594058,50.872094
3,218527,A 4-53,50.873007,-2147483648,no,11.593703,POINT (11.5937031 50.8730073),11.594058,50.872094
4,218528,A 4-53,50.872926,-2147483648,no,11.593707,POINT (11.5937074 50.8729262),11.594058,50.872094


In [313]:
print('--------------------projection-on-the-motorway----------------------')
for i,g in Test_53.iterrows():
    neip = motL.interpolate(motL.project(g['geometry']))
    Test_53.loc[i,"x_mot"] = neip.x
    Test_53.loc[i,"y_mot"] = neip.y
    dist = (g["geometry"].xy[0][0]-neip.x)**2+(g["geometry"].xy[1][0]-neip.y)**2
    Test_53.loc[i,"m_dist"] = dist
print('---------------projection-on-the-motorway-finished------------------')  

print('-----------------calculating-chirality-and-angle--------------------')  
Test_53.loc[:,'angle'] = np.arctan2((Test_53['lat']-Test_53['y_mot']),(Test_53['lon']-Test_53['x_mot']))*180./np.pi
Test_53.loc[:,'angle'] = - Test_53.loc[:,'angle']
Test_53.loc[:,'tang']  = np.arctan2(metr["metrics"]['deCenter'][1]-Test_53['lat'],metr["metrics"]['deCenter'][0]-Test_53['lon'])*180./np.pi
Test_53.loc[:,'tang']  = 90. - Test_53.loc[:,'tang']
Test_53.loc[Test_53['tang']>180.,'tang'] -= 180.
t = np.abs(Test_53['tang']-Test_53['angle'])
t[t>180.] = 360.-t
Test_53.loc[:,'chirality'] = 1*(t>90)
print('-------------calculating-chirality-and-angle-finished---------------')
Test_53.head()

--------------------projection-on-the-motorway----------------------
---------------projection-on-the-motorway-finished------------------
-----------------calculating-chirality-and-angle--------------------
-------------calculating-chirality-and-angle-finished---------------


,id,id_jun,lat,node_id,destination,lon,geometry,x,y,x_mot,y_mot,m_dist,angle,tang,chirality
0,218436,A 4-53,50.871500,1478289457,B 88,11.593074,POINT (11.5930741 50.8715002),11.594058,50.872094,11.593074,50.871500,0.000000e+00,116.565051,-78.649901,1
1,218495,A 4-53,50.871972,486611,A 4,11.593503,POINT (11.5935028 50.8719723),11.594058,50.872094,12.227476,50.871472,4.019228e-01,-179.954830,-78.673453,1
2,218504,A 4-53,50.871613,1788468207,B 88,11.593551,POINT (11.5935514 50.8716134),11.594058,50.872094,11.593522,50.871684,5.908343e-09,68.030697,-78.658709,1
3,218527,A 4-53,50.873007,-2147483648,no,11.593703,POINT (11.5937031 50.8730073),11.594058,50.872094,11.594071,50.872228,7.426184e-07,-115.262367,-78.718825,0
4,218528,A 4-53,50.872926,-2147483648,no,11.593707,POINT (11.5937074 50.8729262),11.594058,50.872094,11.594043,50.872215,6.186867e-07,-115.280410,-78.715440,0


In [314]:
mapp = folium.Map(location=[50.87211,11.59474],
                  zoom_start = 18
                  )

Feature_Group_Nodes = folium.map.FeatureGroup(name="Nodes", overlay=True, control=True)

for i,g in Test_53.iterrows():
    folium.features.Circle(location=[g['lat'],g['lon']],
                           popup = (str(g['chirality'])),
                           radius = 2,
                           color = "blue",
                           fill_color = "white"
                          ).add_to(Feature_Group_Nodes)
    
Feature_Group_CoM = folium.map.FeatureGroup(name="Center of Mass", overlay=True, control=True)

for i,g in Test_53.iterrows():
    folium.features.Circle(location=[g['y'],g['x']],
                           popup = (str(g['id_jun'])),
                           radius = 2,
                           color = "red",
                           fill_color = "white"
                          ).add_to(Feature_Group_CoM)


Feature_Group_Nodes.add_to(mapp)
Feature_Group_CoM.add_to(mapp)
folium.LayerControl().add_to(mapp)
mapp

# Criteria for Nodeselction

In [315]:
###--------------------------don't-use-global-variables-in-functions-----------------
def Node_Criteria(chi, dist):
    NodeS = Test_53.loc[Test_53['chirality'] == chi]
    NodeS = NodeS.loc[Test_53['m_dist'] != 0]
    NodeS = NodeS.loc[Test_53['m_dist'] > dist]
    NodeS = NodeS.sort_values(["m_dist"])
    NodeS = NodeS.head(2)
    return NodeS

In [316]:
Node_Criteria(1, 1.0e-08)

,id,id_jun,lat,node_id,destination,lon,geometry,x,y,x_mot,y_mot,m_dist,angle,tang,chirality
57,218686,A 4-53,50.872279,1789167777,B 88,11.595069,POINT (11.5950693 50.8722785),11.594058,50.872094,11.595004,50.872401,1.931240e-08,61.931860,-78.699619,1
17,218552,A 4-53,50.871654,1788468209,B 88,11.593824,POINT (11.5938239 50.8716538),11.594058,50.872094,11.593769,50.871786,2.049387e-08,67.575108,-78.662749,1


# Plot Nodeselection

In [317]:
mapp = folium.Map(location=[50.87211,11.59474],
                  zoom_start = 18
                  )

Feature_Group_Nodes = folium.map.FeatureGroup(name="Nodes", overlay=True, control=True)

for i in [0,1]:
    NodeS = Test_53.loc[Test_53['chirality'] == i]
    NodeS = NodeS.loc[Test_53['m_dist'] != 0]
    NodeS = NodeS.loc[Test_53['m_dist'] > 1.0e-08]
    NodeS = NodeS.sort_values(["m_dist"])
    NodeS = NodeS.head(2) 
    for i,g in NodeS.iterrows():
        folium.features.Circle(location=[g['lat'],g['lon']],
                               popup = (str(g['angle'])),
                               radius = 2,
                               color = "blue",
                               fill_color = "white"
                              ).add_to(Feature_Group_Nodes)
        
    for i,g in Test_53.iterrows():
        folium.features.Circle(location=[g['y'],g['x']],
                               popup = (str(g['id_jun'])),
                               radius = 2,
                               color = "red",
                               fill_color = "white"
                              ).add_to(Feature_Group_CoM)


Feature_Group_Nodes.add_to(mapp)
Feature_Group_CoM.add_to(mapp)
folium.LayerControl().add_to(mapp)
mapp

# Create Endlist of Nodes 

In [318]:
Motorway_Nodes = pd.DataFrame()
for i in [0,1]:
    NodeS = Test_53.loc[Test_53['chirality'] == i]
    NodeS = NodeS.loc[Test_53['m_dist'] != 0]
    NodeS = NodeS.loc[Test_53['m_dist'] > 1.0e-08]
    NodeS = NodeS.sort_values(["m_dist"])
    NodeS = NodeS.head(2)
    if i == 0:                                                                    #what criteria do we need??
        NodeS.loc[:,"Junction"] = NodeS["id_jun"].apply(lambda x: x + "_entry")
    if i == 1:
        NodeS.loc[:,"Junction"] = NodeS["id_jun"].apply(lambda x: x + "_exit")
    #NodeS = NodeS[["id_jun","node_id","lat","lon","chirality","angle"]]
    Motorway_Nodes = pd.concat([Motorway_Nodes, NodeS], ignore_index=True)

In [319]:
Motorway_Nodes

,id,id_jun,lat,node_id,destination,lon,geometry,x,y,x_mot,y_mot,m_dist,angle,tang,chirality,Junction
0,218538,A 4-53,50.872201,-2147483648,A 4,11.593753,POINT (11.5937529 50.8722008),11.594058,50.872094,11.593797,50.872101,1.196781e-08,-113.485934,-78.685230,0,A 4-53_entry
1,218618,A 4-53,50.872533,1789167786,B 88,11.594419,POINT (11.5944186 50.8725328),11.594058,50.872094,11.594473,50.872416,1.646476e-08,-114.774132,-78.704854,0,A 4-53_entry
2,218686,A 4-53,50.872279,1789167777,B 88,11.595069,POINT (11.5950693 50.8722785),11.594058,50.872094,11.595004,50.872401,1.931240e-08,61.931860,-78.699619,1,A 4-53_exit
3,218552,A 4-53,50.871654,1788468209,B 88,11.593824,POINT (11.5938239 50.8716538),11.594058,50.872094,11.593769,50.871786,2.049387e-08,67.575108,-78.662749,1,A 4-53_exit


# Save csv File

In [320]:
#Motorway_Nodes.to_csv(baseDir + "Motorway_Nodes.csv")